In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 137kB/s 
     |████████████████████████████████| 6.8MB 41.7MB/s 
     |████████████████████████████████| 983kB 59.5MB/s 
     |████████████████████████████████| 266kB 54.0MB/s 
     |████████████████████████████████| 1.3MB 47.2MB/s 
     |████████████████████████████████| 1.2MB 43.7MB/s 
     |████████████████████████████████| 471kB 52.7MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
     |████████████████████████████████| 2.9MB 48.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=c9cf5c744ca948f98590f2e8e5f86e4d105e60060e8a3d54ec525f47f9b1c747
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=45d503adeb30e55f30273d6fd80e9c722111c26134803f6a745af6949499dcd3
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [ ]:
df_fake=pd.read_csv('/content/drive/MyDrive/datasets hackathon/Fake.csv')
df_real=pd.read_csv('/content/drive/MyDrive/datasets hackathon/True.csv')


In [ ]:

df_fake['label'] = '0'

df_real['label'] = '1' 


In [ ]:
df_fake.drop(['subject','date'], axis = 1,inplace=True)
df_real.drop(['subject','date'], axis = 1,inplace=True) 

In [ ]:
frames = [df_fake, df_real]

result = pd.concat(frames)

In [ ]:
result




,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [ ]:
result['text']=result['text']+result['title']

In [ ]:
from sklearn.utils import shuffle
result = shuffle(result)
result.reset_index(inplace=True, drop=True)

In [ ]:
result

,title,text,label
0,LOCAL SHERIFF Escorting School Kids Not Allowe...,Because shootings never happen in a gun-free t...,0
1,British PM May outraged at North Korea's 'reck...,LONDON (Reuters) - Prime Minister Theresa May ...,1
2,Senate's CIA torture report will go into Obama...,WASHINGTON (Reuters) - President Barack Obama’...,1
3,ALL ABOARD THE SHARIA LAW TRAIN: Germany Annou...,Well that didn t take long In the trains betwe...,0
4,Indonesia questions fireworks factory owner af...,"KOSAMBI, Indonesia (Reuters) - Indonesian poli...",1
...,...,...,...
44893,Influential Shi'ite cleric Sadr says Americans...,BAGHDAD (Reuters) - An influential Shi’ite cle...,1
44894,"Trump to meet with CIA, FBI and DNI chiefs on ...",WASHINGTON (Reuters) - Donald Trump planned to...,1
44895,BILL CLINTON Goes At It With TRUMP Supporter I...,Bill Clinton has lost it! He goes after a Trum...,0
44896,Pennsylvania attorney general resigns followin...,(Reuters) - Pennsylvania Attorney General Kath...,1


In [ ]:
data_train= result.iloc[:30000,:]
data_test = result.iloc[30001:,:]


In [ ]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='text', label_columns='label',
                   val_df = data_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')


preprocessing train...
language: en
train sequence lengths:
	mean : 418
	95percentile : 914
	99percentile : 1515


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 417
	95percentile : 919
	99percentile : 1533


In [ ]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=2)




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
5000/5000 [==============================] - 1793s 356ms/step - loss: 0.1081 - accuracy: 0.9690 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 2/2
5000/5000 [==============================] - 1792s 357ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 2.5518e-04 - val_accuracy: 1.0000


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
predictor.save('/content/drive/My Drive/distilbert')

In [ ]:
data = ['this movie was really bad. acting was also bad. I will not watch again',
        'the movie was really great. I will see it again', 'another great movie. must watch to everyone']

In [ ]:
predictor.predict(data)

['0', '0', '0']

In [ ]:
predictor.get_classes()

['0', '1']

In [ ]:
predictor.predict(data, return_proba=True)

array([[9.9993300e-01, 6.6942695e-05],
       [9.9989736e-01, 1.0264792e-04],
       [9.9947673e-01, 5.2328338e-04]], dtype=float32)